In [ ]:
# Standard library
from os.path import join

# Third-party
import corner
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.celestialmechanics import OrbitalParams
from thejoker.plot import plot_rv_curves, _truth_color

plt.style.use('../thejoker/thejoker.mplstyle')

from figurehelpers import samples_units, make_rv_curve_figure, apw_corner

In [ ]:
dpi = 256

In [ ]:
# experiment number
e_number = 1
e_name = 'validation'

In [ ]:
data_filename = "../data/experiment{}.h5".format(e_number)

## read the data:

In [ ]:
# read the data
with h5py.File(data_filename, 'r') as f:
    data = RVData.from_hdf5(f)
    truth_pars = OrbitalParams.from_hdf5(f['truth'])
    truth_vec = truth_pars.pack(plot_transform=True, units=samples_units)[0]

# read the samples from fixing the jitter
with h5py.File(join(paths.cache, 'experiment{}-fixed-jitter.h5'.format(e_number)), 'r') as g:
    pars1 = OrbitalParams.from_hdf5(g)
    samples1 = pars1.pack(plot_transform=True, units=samples_units)
    samples1[:,3] = (samples1[:,3] - 360*((data.t_offset/pars1._P) % 1.)) % (360) # HACK

# read the samples from sampling over the jitter
with h5py.File(join(paths.cache, 'experiment{}-sample-jitter.h5'.format(e_number)), 'r') as g:
    pars2 = OrbitalParams.from_hdf5(g)
    samples2 = pars2.pack(plot_transform=True, units=samples_units)
    samples2[:,3] = (samples2[:,3] - 360*((data.t_offset/pars2._P) % 1.)) % (360)

print("{} samples survived (a)".format(samples1.shape[0]))
print("{} samples survived (b)".format(samples2.shape[0]))

In [ ]:
t_grid = np.linspace(data.t.mjd.min(), data.t.mjd.max(), 1024)
truth_rv = truth_pars.rv_orbit(0).generate_rv_curve(t_grid)

plt.plot(t_grid, truth_rv.to(samples_units['v0']).value, linestyle='--', 
         marker='', linewidth=1, alpha=0.9, color=_truth_color)
data.plot(ax=plt.gca(), rv_unit=samples_units['v0'], markersize=6)

In [ ]:
fig = make_rv_curve_figure(data, [pars1, pars2], truth_pars=truth_pars,
                           units=samples_units, rv_lim=(21, 67))
fig.axes[0].set_title("Experiment {}".format(e_number), fontsize=22)

fig.axes[0].text(55700, 60, "(a)", fontsize=18)
fig.axes[1].text(55700, 60, "(b)", fontsize=18)

fig.tight_layout()
fig.savefig(join(paths.figures, '{}-rv-curves.pdf'.format(e_name)), dpi=dpi)

## Corner plots

In [ ]:
# Make a corner plot of all samples
_med_v0 = np.median(samples1[:,-1])
_mad_v0 = np.median(np.abs(samples1[:,-1] - _med_v0))
ranges = [(2.55, 6.45), (-0.025, 1.025), (0,360), (0,360), (-7.25, 7.25),
          (5, 25), (_med_v0 - 5*_mad_v0, _med_v0 + 5*_mad_v0)]
labels = OrbitalParams.get_labels(samples_units)

corner_style = dict(alpha=0.5, markersize=2., bins=16, color='#555555')

In [ ]:
# remove jitter from (a) plot
s_idx = 4
fig = apw_corner(np.delete(samples1, s_idx, axis=1), range=np.delete(ranges, s_idx, axis=0),
                 truths=np.delete(truth_vec, s_idx), labels=np.delete(labels, s_idx),
                 **corner_style)
fig.subplots_adjust(left=0.08, bottom=0.08)
fig.suptitle("Experiment {}(a): fixed jitter".format(e_number), fontsize=26)
fig.savefig(join(paths.figures, '{}-corner-a.pdf'.format(e_name)), dpi=dpi)

In [ ]:
fig = apw_corner(samples2, range=ranges, truths=truth_vec, labels=labels, **corner_style)
fig.subplots_adjust(left=0.08, bottom=0.08)
fig.suptitle("Experiment {}(b): sample jitter".format(e_number), fontsize=26)
fig.savefig(join(paths.figures, '{}-corner-b.pdf'.format(e_name)), dpi=dpi)